In [1]:
# !pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import wandb
from tqdm import tqdm
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.colors as mcolors

In [3]:
class wandb_results:
    def __init__(self, project_id, wandb_username="eilamshapira"):
        self.api = wandb.Api(timeout=60)
        self.project_id = project_id
        self.wandb_username = wandb_username

    def get_sweep_results(self, sweep_id, metric="accuracy_all", best_epoch=False, get_servers=False, reset_api=False, read_csv_if_exist=True, save_to_csv=True):
        if reset_api:
            self.reset_api()

        print(f"Download {sweep_id=} data...")
        runs = self.api.sweep(f"{self.wandb_username}/{self.project_id}/{sweep_id}").runs
        n_runs = len(runs)
        path = f"sweeps_csvs/{sweep_id}_{n_runs}.csv"
        if read_csv_if_exist and os.path.exists(path):
            return pd.read_csv(path, index_col=0)
        summary_list, config_list, name_list = [], [], []
        for run in tqdm(runs):
            summary_list.append(run.summary._json_dict)
            config_list.append(
                {k: v for k,v in run.config.items()
                  if not k.startswith('_')})
            name_list.append(run.name)

        runs_df = pd.DataFrame({
            "summary": summary_list,
            "config": config_list,
            "name": name_list
            })
        config_cols = pd.json_normalize(runs_df['config'])
        config_cols.columns = [f"config_{c}" for c in config_cols.columns]
        summary_cols = pd.json_normalize(runs_df['summary'])
        runs_df = pd.concat([runs_df, config_cols, summary_cols], axis=1)
        runs_df.drop(['config', 'summary'], axis=1, inplace=True)
        hpt = [c for c in config_cols.columns if c not in ["config_seed", "config_run_hash"]]
        if save_to_csv: runs_df.to_csv(path)
        return runs_df

    def get_sweeps_results(self, sweeps, metric="accuracy_all", best_epoch=False, get_servers=False,  read_csv_if_exist=True, save_to_csv=True):
        print("Total number of sweeps:", len(sweeps))
        j = pd.concat([self.get_sweep_results(sweep, metric=metric, best_epoch=best_epoch,  get_servers=get_servers, save_to_csv=save_to_csv, read_csv_if_exist=read_csv_if_exist) for sweep in sweeps])
        j = j.reset_index(drop=True)
        return j

    def reset_api(self):
        self.api = wandb.Api()

In [5]:
# from read_wandb import wandb_results
api = wandb_results("NLP2024_PROJECT_AliMassalha", wandb_username="technions") #1cfb48ee50f664359729acbc119118a7525b4242

BASE_METRIC = "accuracy_per_mean_user_and_bot"

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# def result_metric(sweeps, group_name, drop_list=[0], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
#     df = api.get_sweeps_results(sweeps, metric=metric)
#
#     config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
#     HPT_cols = [col for col in config_cols if df[col].nunique() > 1]
#     print(HPT_cols)
#     if drop_HPT:
#         df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
#         HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]
#
#     grouped = df.groupby([c for c in HPT_cols if c != "config_seed"])
#
#     mean_df = grouped.mean()
#     std_df = grouped.std()
#     if epoch=="best":
#         best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
#     else:
#         best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)
#
#     result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
#     means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
#     stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())
#
#
#     df_cols = {'mean': means, 'std': stds, 'values': result.values}
#     if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")
#
#     df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))
#
#     summary_df = pd.DataFrame(df_cols, index=best_col.index)
#     for d in drop_list:
#         if d in summary_df.index:
#             summary_df=summary_df.drop(d)
#     if len(summary_df.index.names) == 1:
#         return summary_df.rename_axis(group_name)
#     else:
#         return summary_df
#
# def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
#     bootstrapped_means = []
#     for _ in range(n_bootstrap):
#         sample = np.random.choice(data, size=len(data), replace=True)
#         bootstrapped_means.append(np.mean(sample))
#     lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
#     upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
#     return lower_bound, upper_bound

In [ ]:
def result_metric(sweeps, group_name, drop_list=[0], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
    df = api.get_sweeps_results(sweeps, metric=metric)

    config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
    HPT_cols = [col for col in config_cols if df[col].nunique() > 1]
    print(HPT_cols)
    if drop_HPT:
        df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
        HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]

    # Remove non-numeric columns before computing mean and std
    numeric_cols = df.select_dtypes(include=np.number).columns
    df_numeric = df[numeric_cols]

    grouped = df_numeric.groupby([c for c in HPT_cols if c != "config_seed"])

    mean_df = grouped.mean()
    std_df = grouped.std()

    # Re-add non-numeric columns before computing best_col
    for col in config_cols:
        if col not in mean_df.columns:
            mean_df[col] = df[col]

    if epoch=="best":
        best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
    else:
        best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)

    result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
    means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
    stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())


    df_cols = {'mean': means, 'std': stds, 'values': result.values}
    if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")

    df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))

    summary_df = pd.DataFrame(df_cols, index=best_col.index)
    for d in drop_list:
        if d in summary_df.index:
            summary_df=summary_df.drop(d)
    if len(summary_df.index.names) == 1:
        return summary_df.rename_axis(group_name)
    else:
        return summary_df

def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(sample))
    lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
    return lower_bound, upper_bound


# For HyperParameterTuning

For every configuration that you test in the sweep, you will receive in the table the average, standard deviation, all the values obtained for the different seed values, and also the confidence interval within which the result is located at a confidence level of 95%.

When epoch="best" is defined, you can check in which epoch the best result is obtained. If epoch=5 is defined, you will receive the result obtained for epoch number 5.

You can test multiple sweeps simultaneously by entering them into the list found in the first element of the function result_metric.

In [ ]:
# Create the directory if it doesn't exist
directory = 'sweeps_csvs'
if not os.path.exists(directory):
    os.makedirs(directory)

sweep_results = result_metric(["2kmkkxka"], "basic_nature", drop_HPT=False, epoch="best")
sweep_results

# Result for a specific epoch

In [ ]:
sweep_results = result_metric(["2kmkkxka"], "basic_nature", drop_HPT=False, epoch=1)
sweep_results

Total number of sweeps: 1
Download sweep_id='px13bj47' data...
['config_seed', 'config_layers', 'config_hidden_dim']


mean       std  \
config_layers config_hidden_dim                       
2             32                 0.822903  0.004485   
              64                 0.821078  0.004435   
4             32                 0.821501  0.003652   
              64                 0.825733  0.002764   

                                                                            values  \
config_layers config_hidden_dim                                                      
2             32                 [0.8202714466298694, 0.8280817740751573, 0.820...   
              64                 [0.8224877120590862, 0.8246368273885568, 0.816...   
4             32                 [0.8179865378516612, 0.8252760405650429, 0.821...   
              64                 [0.8226790369866819, 0.8280626878370055, 0.826...   

                                                                       CI  
config_layers config_hidden_dim                                            
2             32                 (0.8202714466298694, 0.8280817740751573)  
              64                 (0.8161096617444225, 0.8246368273885568)  
4             32                 (0.8179865378516612, 0.8252760405650429)  
              64                 (0.8226790369866818, 0.8280626878370055)